<a href="https://colab.research.google.com/github/manoj-73/Deep-fake-detection-using-CNN/blob/main/Deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install unrar tool
!apt-get install unrar > /dev/null

# Upload file
from google.colab import files
uploaded = files.upload()  # Upload `deepfake.rar`

# Extract
!unrar x deepfake.rar


Saving deepfake.rar to deepfake (1).rar

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from deepfake.rar

Creating    deepfake                                                  OK
Extracting  deepfake/1(a).jpg                                              0%  OK 
Extracting  deepfake/1(o).jpeg                                             1%  OK 
Extracting  deepfake/10(a).jpg                                             1%  OK 
Extracting  deepfake/10(o).jpeg                                            1%  OK 
Extracting  deepfake/100(a).jpg                                            1%  OK 
Extracting  deepfake/100(o).jpg                                            1%  OK 
Extracting  deepfake/101(a).jpg                                            1%  OK 
Extracting  deepfake/101(o).jpg                                            1%  OK 
Extracting  deepfake/102(a).jpg              

In [ ]:
import os
import shutil

# Original image folder
source_dir = 'deepfake'

# Target folders
target_deepfake = 'dataset/deepfake'
target_human = 'dataset/human'

os.makedirs(target_deepfake, exist_ok=True)
os.makedirs(target_human, exist_ok=True)

# Move images to class folders
for file in os.listdir(source_dir):
    if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
        if '(a)' in file:
            shutil.copy(os.path.join(source_dir, file), target_deepfake)
        elif '(o)' in file:
            shutil.copy(os.path.join(source_dir, file), target_human)


In [ ]:
print("Deepfake images:", len(os.listdir('dataset/deepfake')))
print("Human images   :", len(os.listdir('dataset/human')))


Deepfake images: 361
Human images   : 359


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (128, 128)
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    'dataset',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    'dataset',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)


Found 577 images belonging to 2 classes.
Found 143 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 256ms/step - accuracy: 0.5225 - loss: 0.9638 - val_accuracy: 0.5035 - val_loss: 0.6932
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.4825 - loss: 0.6946 - val_accuracy: 0.5035 - val_loss: 0.6967
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.4773 - loss: 0.6991 - val_accuracy: 0.5035 - val_loss: 0.6931
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.4697 - loss: 0.6932 - val_accuracy: 0.4965 - val_loss: 0.6932
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.5071 - loss: 0.6932 - val_accuracy: 0.4965 - val_loss: 0.6932
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5163 - loss: 0.6930 - val_accuracy: 0.4965 - val_loss: 0.6932
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.5250 - loss: 0.6928 - val_accuracy: 0.4965 - val_loss: 0.6932
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.5275 - loss: 0.6928 - val_accuracy: 0.4965 - val_loss: 

In [ ]:
model.save("deepfake_detector.h5")


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload a new test image


Saving deepfake.rar to deepfake (2).rar


In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from google.colab import files

# Upload an image
uploaded = files.upload()

# Get uploaded file name
img_path = list(uploaded.keys())[0]

# Try loading image using OpenCV
img = cv2.imread(img_path)

# Handle read errors
if img is None:
    raise ValueError("❌ The uploaded file is not a valid image (JPG/PNG). Please try again.")

# Preprocess image
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
img = cv2.resize(img, (128, 128))           # Resize
img_array = img.astype("float32") / 255.0   # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Load model
model = load_model("deepfake_detector.h5")

# Predict
prediction = model.predict(img_array)[0][0]

# Show result
if prediction > 0.5:
    print(f"{img_path} → 🤖 Deepfake ({prediction:.2f})")
else:
    print(f"{img_path} → 🧑 Real Human ({1 - prediction:.2f})")


Saving download (3.1).jpg to download (3.1).jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
download (3.1).jpg → 🤖 Deepfake (0.50)
